In [10]:
# Python 3.7.6
import sys
print(sys.version)

3.6.4 (v3.6.4:d48eceb, Dec 19 2017, 06:54:40) [MSC v.1900 64 bit (AMD64)]


In [11]:
from math import pi
import numpy as np
import pandas as pd
import copy
import itertools as I
import math
from random import shuffle
import shutil, os
import matplotlib.pylab as plt
import seaborn as sns
import scipy.stats

In [12]:
### List of all possible 4-trading periods combinations.
# Needed to pay subjects.
rounds = [i for i in range(1, 12)]
combos = []
for subset in I.combinations(rounds, 4):
    combos.append(subset)
len(combos)
shuffle(combos)
index = [str(i) for i in range(1, 331)]
df = pd.DataFrame(combos, columns = ['Selected period 1','Selected period 2', 'Selected period 3', 'Selected period 4'],
                  index=index)
df.index.name='Label'
df.to_csv('output/periodCombos.csv')

In [13]:
# Import Raw Data
myData = pd.read_csv("input/rawData.csv")

In [14]:
# Function. 
# Input: No-negative integer (to denote a row in the dataset). 
# Output: text label from ['Base', 'Harsh'].
# getTreatment generates a text label for the treatment ('Base' for baseline, and 'Harsh' for the cap treatment)
# it looks at a given row in the dataset, and if the observation belongs to the baseline (cap) treatment, the code returns the 
# appropriate label
def getTreatment (row):
    for t in ['Base', 'Harsh']:
        if row[t]:
            return t

In [15]:
# Function. 
# Input: No-negative integer (to denote a row in the dataset). 
# Output: text label from ['onePackage', 'twoPackages', 'threePackages', 'fourPackages', 'noPackage'].
# getOffers generates a text label for the number of packages submitted by the seller. 
# It looks at a giver row in the dataset, and if the observation contains one, two, three, or four packages submitted by the
# seller, it returns the corresponding label ('onePackage', 'twoPackages', 'threePackages', 'fourPackages'). 
# If the seller decided not to offer the function returns 'noPackage'
def getOffers (row):
    for a, b in zip(['offerOne', 'offerTwo', 'offerThree', 'offerFour'],
                    ['onePackage', 'twoPackages', 'threePackages', 'fourPackages']):
        if row['offerAny']:
            if row[a]:
                return b
        else:
            return 'noPackage'

In [16]:
# Get a label indicating the treatment
myData['treatment'] = myData.apply (lambda row: getTreatment (row),axis=1)
# Get a label indicating the number of packages offered
myData['offers'] = myData.apply (lambda row: getOffers (row),axis=1)

In [17]:
# remove automatically-generated row index. 
myData = myData.reset_index()

In [18]:
# Get a dummy per session. In case needed.
dSessions = pd.get_dummies(myData['sessionCode'], prefix='', prefix_sep = "", drop_first = False)
myData = myData.join(dSessions)

In [19]:
def getRents(datos, row):
    df = copy.deepcopy(datos)
    ll = ['oneQA', 'oneQB', 'onePrice', 'twoQA', 'twoQB', 'twoPrice', 'threeQA', 'threeQB', 'threePrice', 'fourQA',
          'fourQB', 'fourPrice']
    # Create a list with each package offered by the seller.
    if df['offerOne'][row] == 1:
        lPacks = ['P1']
    elif df['offerTwo'][row] == 1:
        lPacks = ['P1', 'P2']
    elif df['offerThree'][row] == 1:
        lPacks = ['P1', 'P2', 'P3']
    elif df['offerFour'][row] == 1:
        lPacks = ['P1', 'P2', 'P3', 'P4']
    elif df['offerAny'][row] == 0:
        lPacks = ['P1']

    cols = []
    for i, a in enumerate(lPacks):
        cols.append(ll[i * 3: (i + 1) * 3])
    # Given the row we are currently looking at, take packages characteristics (q's and prices)
    inp = []
    for i, c in enumerate(cols):
        jj = df[cols[i]].values.tolist()[row]
        inp.append(jj)
        #[[qa, qb, pr], [qa, qb, pr], ...]
        
    # Dictionary {'P1': [qa, qb, pr], ...}
    CONTRACT = {k: None for k in lPacks}
    for i, j in enumerate(lPacks):
        CONTRACT[j] = inp[i]

    # For each package, calculate the implied rents for the ij-type.
    # ij = {'P1': Rij1, 'P2': Rij2, 'P3': Rij3, 'P4':Rij4}
    HH = {k: 15 * math.sqrt(v[0]) + 15 * math.sqrt(v[1]) - v[2] for k, v in zip(lPacks, inp)}
    HL = {k: 15 * math.sqrt(v[0]) + 10 * math.sqrt(v[1]) - v[2] for k, v in zip(lPacks, inp)}
    LH = {k: 10 * math.sqrt(v[0]) + 15 * math.sqrt(v[1]) - v[2] for k, v in zip(lPacks, inp)}
    LL = {k: 10 * math.sqrt(v[0]) + 10 * math.sqrt(v[1]) - v[2] for k, v in zip(lPacks, inp)}

    # Document choices
    # C =  { P1: [HH, HL], P2: [LL]...}
    C = {pack: [] for pack in lPacks}
    # Document consumer surplus derived by the package each type would buy and the quantities actually purchased by each type.
    R = {d: [] for d in ['RHH','RHL','RLH','RLL',
                         'AHH','BHH','PHH','AHL','BHL','PHL','ALH','BLH','PLH','ALL','BLL','PLL']}
    # Package with the largest info rents for the HH-type
    mhh = max(HH, key = HH.get)
    vhh = HH[mhh]
    # Document the rents.
    # If rents are strictly positive, then HH buys the package with largest info rents.
    if vhh >= 0:
        for i, j in enumerate(['AHH','BHH','PHH']):
            R[j].append( CONTRACT[mhh][i] )
        R['RHH'].append( max(HH.values()) )
        C[mhh].append('HH')
    # Else she buys nothing and gains 0 points.
    else:
        for i, j in enumerate(['AHH','BHH','PHH']):
            R[j].append(None)
        R['RHH'].append(0)

    mhl = max(HL, key=HL.get)
    vhl = HL[mhl]
    if vhl >= 0:
        for i, j in enumerate(['AHL','BHL','PHL']):
            R[j].append(CONTRACT[mhl][i])
        R['RHL'].append(max(HL.values()))
        C[mhl].append('HL')
    else:
        for i, j in enumerate(['AHL','BHL','PHL']):
            R[j].append(None)
        R['RHL'].append(0)

    mlh = max(LH, key=LH.get)
    vlh = LH[mlh]
    if vlh >= 0:
        for i, j in enumerate(['ALH','BLH','PLH']):
            R[j].append( CONTRACT[mlh][i] )
        R['RLH'].append( max(LH.values()) )
        C[mlh].append('LH')
    else:
        for i, j in enumerate(['ALH','BLH','PLH']):
            R[j].append(None)
        R['RLH'].append(0)

    mll = max(LL, key=LL.get)
    vll = LL[mll]
    if vll >= 0:
        for i, j in enumerate(['ALL','BLL','PLL']):
            R[j].append( CONTRACT[mll][i] )
        R['RLL'].append( max(LL.values()) )
        C[mll].append('LL')
    else:
        for i, j in enumerate(['ALL','BLL','PLL']):
            R[j].append(None)
        R['RLL'].append(0)

    # Now get expected profit
    #Distribution of types. 
    B = [0.1, 0.4, 0.1]
    # Need this is step to avoid 'None' values.
    a = None
    b = None
    c = None
    d = None
    if R['ALL'][0] == None:
        a = 0
    else:
        a = R['PLL'][0] - 0.002 * (R['ALL'][0] ** 2 + R['BLL'][0] ** 2)
    if R['ALH'][0] == None:
        b = 0
    else:
        b =  R['PLH'][0] - 0.002 * (R['ALH'][0] ** 2 + R['BLH'][0] ** 2)
    if R['ALL'][0] == None:
        c = 0
    else:
        c = R['PHL'][0] - 0.002 * (R['AHL'][0] ** 2 + R['BHL'][0] ** 2)
    if R['ALH'][0] == None:
        d = 0
    else:
        d = R['PHH'][0] - 0.002 * (R['AHH'][0] ** 2 + R['BHH'][0] ** 2)

    R['EPI'] = [B[0]*a + B[1]*b + B[1]*c + B[2]*d]

    # Get welfare (profit + each type's consumer surplus)
    R['EW'] = [ R['EPI'][0] + B[2] * R['RHH'][0] + B[1] * R['RHL'][0] + B[1] * R['RLH'][0] + B[0] * R['RLL'][0]]


    Sizes = {k: v[0] + v[1] for k, v in zip(lPacks, inp)}
    # reverse = True orders larger to smaller.
    sort = sorted(Sizes, key=Sizes.get, reverse=True)

    if df['offerAny'][row] == 0:
        for i in ['A1', 'B1', 'P1', 'A2', 'B2', 'P2', 'A3', 'B3', 'P3', 'A4', 'B4', 'P4']:
            R[i] = None
    else:
        if df['offerOne'][row] == 1:
            R['A1'] = CONTRACT[sort[0]][0]
            R['B1'] = CONTRACT[sort[0]][1]
            R['P1'] = CONTRACT[sort[0]][2]

            for i in ['A2', 'B2', 'P2', 'A3', 'B3', 'P3', 'A4', 'B4', 'P4']:
                R[i] = None

        if df['offerTwo'][row] == 1:
            R['A1'] = CONTRACT[sort[0]][0]
            R['B1'] = CONTRACT[sort[0]][1]
            R['P1'] = CONTRACT[sort[0]][2]

            R['A2'] = CONTRACT[sort[1]][0]
            R['B2'] = CONTRACT[sort[1]][1]
            R['P2'] = CONTRACT[sort[1]][2]

            for i in ['A3', 'B3', 'P3', 'A4', 'B4', 'P4']:
                R[i] = None

        if df['offerThree'][row] == 1:
            R['A1'] = CONTRACT[sort[0]][0]
            R['B1'] = CONTRACT[sort[0]][1]
            R['P1'] = CONTRACT[sort[0]][2]

            R['A2'] = CONTRACT[sort[1]][0]
            R['B2'] = CONTRACT[sort[1]][1]
            R['P2'] = CONTRACT[sort[1]][2]

            R['A3'] = CONTRACT[sort[2]][0]
            R['B3'] = CONTRACT[sort[2]][1]
            R['P3'] = CONTRACT[sort[2]][2]

            for i in ['A4', 'B4', 'P4']:
                R[i] = None

        if df['offerFour'][row] == 1:
            R['A1'] = CONTRACT[sort[0]][0]
            R['B1'] = CONTRACT[sort[0]][1]
            R['P1'] = CONTRACT[sort[0]][2]

            R['A2'] = CONTRACT[sort[1]][0]
            R['B2'] = CONTRACT[sort[1]][1]
            R['P2'] = CONTRACT[sort[1]][2]

            R['A3'] = CONTRACT[sort[2]][0]
            R['B3'] = CONTRACT[sort[2]][1]
            R['P3'] = CONTRACT[sort[2]][2]

            R['A4'] = CONTRACT[sort[3]][0]
            R['B4'] = CONTRACT[sort[3]][1]
            R['P4'] = CONTRACT[sort[3]][2]

    # Identify strategy
    C = {k: v for k, v in C.items() if v != []}
    if len(C) == 1:
        if ['HH'] in C.values():
            R['Strategy'] = 'A1'
        elif ['HH', 'HL'] in C.values():
            R['Strategy'] = 'A2'
        elif ['HH', 'LH'] in C.values():
            R['Strategy'] = 'A3'
        elif ['HH', 'HL', 'LH'] in C.values():
            R['Strategy'] = 'A4'
        elif ['HH', 'HL', 'LH', 'LL'] in C.values():
            R['Strategy'] = 'A5'
        else:
            R['Strategy'] = 'CHECK'
    if len(C) == 2:
        if (['HH'] in C.values() and ['HL'] in C.values()):
            R['Strategy'] = 'B1'
        elif (['HH'] in C.values() and ['LH'] in C.values()):
            R['Strategy'] = 'B2'
        elif (['HH'] in C.values() and ['HL', 'LH'] in C.values()):
            R['Strategy'] = 'B3'
        elif (['HH', 'HL'] in C.values() and ['LH', 'LL'] in C.values()):
            R['Strategy'] = 'B4'
        elif (['HH'] in C.values() and ['HL', 'LH', 'LL'] in C.values()):
            R['Strategy'] = 'B5'
        elif (['HH', 'LH'] in C.values() and ['HL', 'LL'] in C.values()):
            R['Strategy'] = 'B6'
        elif (['HH', 'HL', 'LH'] in C.values() and ['LL'] in C.values()):
            R['Strategy'] = 'B7'
        elif (['HH', 'LH', 'LL'] in C.values() and ['HL'] in C.values()):
            R['Strategy'] = 'B8'
        elif (['HH', 'HL', 'LL'] in C.values() and ['LH'] in C.values()):
            R['Strategy'] = 'B9'
        elif (['HH', 'LH'] in C.values() and ['LL'] in C.values()):
            R['Strategy'] = 'B10'
        elif (['HH', 'LH'] in C.values() and ['HL'] in C.values()):
            R['Strategy'] = 'B11'
        elif (['HH', 'HL'] in C.values() and ['LH'] in C.values()):
            R['Strategy'] = 'B12'
        else:
            R['Strategy'] = 'CHECK'
    if len(C) == 3:
        if (['HH'] in C.values() and ['HL'] in C.values() and ['LH'] in C.values()):
            R['Strategy'] = 'C1'
        elif (['HH'] in C.values() and ['HL'] in C.values() and ['LH', 'LL'] in C.values()):
            R['Strategy'] = 'C2'
        elif (['HH'] in C.values() and ['LH'] in C.values() and ['HL', 'LL'] in C.values()):
            R['Strategy'] = 'C3'
        elif (['HH'] in C.values() and ['HL', 'LH'] in C.values() and ['LL'] in C.values()):
            R['Strategy'] = 'C4'
        elif (['HH', 'HL'] in C.values() and ['LH'] in C.values() and ['LL'] in C.values()):
            R['Strategy'] = 'C5'
        elif (['HH', 'LH'] in C.values() and ['HL'] in C.values() and ['LL'] in C.values()):
            R['Strategy'] = 'C6'
        elif (['HH', 'LL'] in C.values() and ['HL'] in C.values() and ['LH'] in C.values()):
            R['Strategy'] = 'C7'
        else:
            R['Strategy'] = 'CHECK'
    if len(C) == 4:
        if (['HH'] in C.values() and ['HL'] in C.values() and ['LH'] in C.values() and ['LL'] in C.values()):
            R['Strategy'] = 'D1'
        else:
            R['Strategy'] = 'CHECK'
    if len(C) == 0:
        R['Strategy'] = '0'

    return R

In [20]:
# This is am example of how getRents works
getRents(myData, 7)

{'RHH': [129.40335056557728],
 'RHL': [88.47658670621504],
 'RLH': [95.86233090308048],
 'RLL': [54.93556704371821],
 'AHH': [45],
 'BHH': [67],
 'PHH': [94],
 'AHL': [45],
 'BHL': [67],
 'PHL': [94],
 'ALH': [45],
 'BLH': [67],
 'PLH': [94],
 'ALL': [45],
 'BLL': [67],
 'PLL': [94],
 'EPI': [80.972],
 'EW': [173.14145880464775],
 'A1': 60,
 'B1': 60,
 'P1': 135,
 'A2': 45,
 'B2': 67,
 'P2': 94,
 'A3': 52,
 'B3': 35,
 'P3': 88,
 'A4': None,
 'B4': None,
 'P4': None,
 'Strategy': 'A5'}

In [21]:
# This is am example of how the output from getRents can be transformed into a dataframe
Y = pd.DataFrame(getRents(myData, 7) )
Y = Y[['A1', 'B1', 'P1', 'A2', 'B2', 'P2', 'A3', 'B3', 'P3', 'A4', 'B4', 'P4',
          'RHH', 'RHL', 'RLH', 'RLL', 'EPI', 'EW',
          'AHH', 'BHH', 'PHH', 'AHL', 'BHL', 'PHL', 'ALH', 'BLH', 'PLH', 'ALL', 'BLL', 'PLL', 'Strategy']]
Y

A1  B1   P1  A2  B2  P2  A3  B3  P3    A4  ... AHL BHL  PHL  ALH  BLH  PLH  \
0  60  60  135  45  67  94  52  35  88  None  ...  45  67   94   45   67   94   

   ALL  BLL  PLL  Strategy  
0   45   67   94        A5  

[1 rows x 31 columns]

In [22]:
## apply getRents to all rows in the dataset.
apData = []
for r in range(len(myData)):
    Y = pd.DataFrame(getRents(myData, r) )
    Y = Y[['A1', 'B1', 'P1', 'A2', 'B2', 'P2', 'A3', 'B3', 'P3', 'A4', 'B4', 'P4',
           'RHH', 'RHL', 'RLH', 'RLL', 'EPI', 'EW',
           'AHH', 'BHH', 'PHH', 'AHL', 'BHL', 'PHL', 'ALH', 'BLH', 'PLH', 'ALL', 'BLL', 'PLL', 'Strategy']]
    temp = []
    for row in Y.iterrows():
        index, leData = row
        temp.append(leData.tolist())
    apData.append(temp[0])

In [23]:
apData = pd.DataFrame(apData, columns = ['A1', 'B1', 'P1', 'A2', 'B2', 'P2', 'A3', 'B3', 'P3', 'A4', 'B4', 'P4',
                                         'RHH', 'RHL', 'RLH', 'RLL', 'EPI', 'EW', 'AHH', 'BHH', 'PHH',
                                         'AHL', 'BHL', 'PHL', 'ALH', 'BLH', 'PLH', 'ALL', 'BLL', 'PLL', 'Strategy'])

In [24]:
# Attach the newly generated variables to the dataset for analysis.
myData = pd.concat([myData, apData], axis = 1)

In [25]:
# Function
# Input: 'datos': a pandas dataset. 
#        'row':  No-negative integer (to denote a row in the dataset). 
# Output: No-negative integer denoting how many of the offered packages would actually be purchased.
def getObservedPackages (datos, row):
    df = copy.deepcopy(datos)
    if  df['Strategy'][row] in ['0']:
        return 0
    if  df['Strategy'][row] in ['A1', 'A2', 'A3', 'A4', 'A5']:
        return 1
    elif df['Strategy'][row] in ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12']:
        return 2
    elif  df['Strategy'][row] in ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']:
        return 3
    elif  df['Strategy'][row] in ['D1']:
        return 4

In [26]:
l = []
for i in range(myData.shape[0]):
    Y = getObservedPackages(myData, i)
    l.append(Y)

In [27]:
myData['ObservedPackages'] = l

In [28]:
dOffer = pd.get_dummies(myData['ObservedPackages'], prefix='ObsOffer', prefix_sep = "_", drop_first = False)
myData = myData.join(dOffer)

In [29]:
# Function. 
# Input: 'datos': a pandas dataset. 
#        'row':  No-negative integer (to denote a row in the dataset). 
# Output: Dictionary D
#                    {'serveHH': 1-item list 1 if HH is covered, 0 otherwise, 
#                    'serveHL': 1-item list 1 if HL is covered, 0 otherwise,  
#                    'serveLH': 1-item list 1 if LH is covered, 0 otherwise,  
#                    'serveLL': 1-item list 1 if LL is covered, 0 otherwise}
def getCoverage (datos, row):
    df = copy.deepcopy(datos)
    D = {k: None for k in ['serveHH', 'serveHL', 'serveLH', 'serveLL']}

    if df['Strategy'][row] in ['0']:
        for n in ['serveHH', 'serveHL', 'serveLH', 'serveLL']:
            D[n] = [0]

    elif df['Strategy'][row] in ['A1']:
        for n in ['serveHH']:
            D[n] = [1]
        for n in ['serveHL', 'serveLH', 'serveLL']:
            D[n] = [0]

    elif df['Strategy'][row] in ['A2']:
        for n in ['serveHH', 'serveHL']:
            D[n] = [1]
        for n in ['serveLH', 'serveLL']:
            D[n] = [0]

    elif df['Strategy'][row] in ['A3']:
        for n in ['serveHH', 'serveLH']:
            D[n] = [1]
        for n in ['serveHL', 'serveLL']:
            D[n] = [0]

    elif df['Strategy'][row] in ['A4']:
        for n in ['serveHH', 'serveHL', 'serveLH']:
            D[n] = [1]
        for n in ['serveLL']:
            D[n] = [0]

    elif df['Strategy'][row] in ['A5']:
        for n in ['serveHH', 'serveHL', 'serveLH', 'serveLL']:
            D[n] = [1]

    elif df['Strategy'][row] in ['B1']:
        for n in ['serveHH', 'serveHL']:
            D[n] = [1]
        for n in ['serveLH', 'serveLL']:
            D[n] = [0]

    elif df['Strategy'][row] in ['B2']:
        for n in ['serveHH', 'serveLH']:
            D[n] = [1]
        for n in ['serveHL', 'serveLL']:
            D[n] = [0]

    elif df['Strategy'][row] in ['B3']:
        for n in ['serveHH', 'serveHL', 'serveLH']:
            D[n] = [1]
        for n in ['serveLL']:
            D[n] = [0]

    elif df['Strategy'][row] in ['B4']:
        for n in ['serveHH', 'serveHL', 'serveLH', 'serveLL']:
            D[n] = [1]

    elif df['Strategy'][row] in ['B5']:
        for n in ['serveHH', 'serveHL', 'serveLH', 'serveLL']:
            D[n] = [1]

    elif df['Strategy'][row] in ['B6']:
        for n in ['serveHH', 'serveHL', 'serveLH', 'serveLL']:
            D[n] = [1]

    elif df['Strategy'][row] in ['B7']:
        for n in ['serveHH', 'serveHL', 'serveLH', 'serveLL']:
            D[n] = [1]

    elif df['Strategy'][row] in ['B8']:
        for n in ['serveHH', 'serveHL', 'serveLH', 'serveLL']:
            D[n] = [1]

    elif df['Strategy'][row] in ['B9']:
        for n in ['serveHH', 'serveHL', 'serveLH', 'serveLL']:
            D[n] = [1]

    elif df['Strategy'][row] in ['B10']:
        for n in ['serveHH', 'serveLH', 'serveLL']:
            D[n] = [1]
        for n in ['serveHL']:
            D[n] = [0]

    elif df['Strategy'][row] in ['B11']:
        for n in ['serveHH', 'serveHL', 'serveLH']:
            D[n] = [1]
        for n in ['serveLL']:
            D[n] = [0]

    elif df['Strategy'][row] in ['B12']:
        for n in ['serveHH', 'serveHL', 'serveLH']:
            D[n] = [1]
        for n in ['serveLL']:
            D[n] = [0]

    elif df['Strategy'][row] in ['C1']:
        for n in ['serveHH', 'serveHL', 'serveLH']:
            D[n] = [1]
        for n in ['serveLL']:
            D[n] = [0]

    elif df['Strategy'][row] in ['C2']:
        for n in ['serveHH', 'serveHL', 'serveLH', 'serveLL']:
            D[n] = [1]

    elif df['Strategy'][row] in ['C3']:
        for n in ['serveHH', 'serveHL', 'serveLH', 'serveLL']:
            D[n] = [1]

    elif df['Strategy'][row] in ['C4']:
        for n in ['serveHH', 'serveHL', 'serveLH', 'serveLL']:
            D[n] = [1]

    elif df['Strategy'][row] in ['C5']:
        for n in ['serveHH', 'serveHL', 'serveLH', 'serveLL']:
            D[n] = [1]

    elif df['Strategy'][row] in ['C6']:
        for n in ['serveHH', 'serveHL', 'serveLH', 'serveLL']:
            D[n] = [1]

    elif df['Strategy'][row] in ['C7']:
        for n in ['serveHH', 'serveHL', 'serveLH', 'serveLL']:
            D[n] = [1]

    elif df['Strategy'][row] in ['D1']:
        for n in ['serveHH', 'serveHL', 'serveLH', 'serveLL']:
            D[n] = [1]

    return D

In [30]:
getCoverage(myData, 7)

{'serveHH': [1], 'serveHL': [1], 'serveLH': [1], 'serveLL': [1]}

In [31]:
apData = []
for r in range(len(myData)):
    Y = pd.DataFrame(getCoverage(myData, r) )
    temp = []
    for row in Y.iterrows():
        index, leData = row
        temp.append(leData.tolist())
    apData.append(temp[0])

In [32]:
apData = pd.DataFrame(apData, columns = ['serveHH', 'serveHL', 'serveLH', 'serveLL'])
myData = pd.concat([myData, apData], axis = 1)

In [33]:
myData.to_csv('temp/forReg.csv')